In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def get_test_train_data_from_csv(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):

    df_bots = pd.read_csv(bot_file)
    df_bots["bot"] = 1
    df_bots

    df_hum = pd.read_csv(hum_file)
    df_hum["bot"] = 0
    df_hum
    df_merged = pd.concat([df_bots, df_hum], ignore_index=True, sort=False)
    df_merged

    return df_merged

In [3]:
def get_seesion_cat():
    df = get_test_train_data_from_csv()

    session_cat = {}
    for _,row in df[["session_id","bot"]].iterrows():
        session_cat[row[0]] = (row[1])

    return session_cat

In [4]:
session_catogory = get_seesion_cat()

C:\Users\sudin\AppData\Local\Temp\ipykernel_13036\3902581569.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  session_cat[row[0]] = (row[1])


In [5]:
def generate_mv_data():
    mv_dir = ["../scripts/mv_data","../scripts/mv_data2"]
    np_list = []
    np_ses_list = []

    batch = 0

    for dir in mv_dir:
        for np_file in os.listdir(dir):
            data = np.load(dir +"/"+ np_file)
            ses_id= np_file[:-4]
            for key in data:
                X = data[key][...,np.newaxis]
                np_list.append(X)
                np_ses_list.append(ses_id)

                if len(np_list) > 900:
                    dir_name = "mv_data/batch_" + str(batch)
                    X = np.stack(np_list, axis=0)
                    Y = np.array([[session_catogory[session_id]] for session_id in np_ses_list])
                    os.makedirs(dir_name,exist_ok=True)
                    with open(dir_name + "/ses.txt",'w') as fd:
                        fd.write(str(np_ses_list)[1:-1])
                    np.savez(dir_name + "/data.npz", X=X, Y=Y)
                    np_list = []
                    np_ses_list = []
                    batch += 1
                    print("batch ",batch," saved")

                print(".",end="")
            print("done for :",np_file)
        print("===="*40,"\n",dir)
    
    if len(np_list) > 0:
        dir_name = "mv_data/batch_" + str(batch)
        X = np.stack(np_list, axis=0)
        Y = np.array([[session_catogory[session_id]] for session_id in np_ses_list])
        os.makedirs(dir_name,exist_ok=True)
        with open(dir_name + "/ses.txt",'w') as fd:
            fd.write(str(np_ses_list)[1:-1])
        np.savez(dir_name + "/data.npz", X=X, Y=Y)
        np_list = []
        np_ses_list = []


In [6]:
# import matplotlib.pyplot as plt

# ind = 30
# print(stacked_h[ind,:].shape)
# print(np_ses_list[ind])
# print(session_catogory[np_ses_list[ind]])

# def show_2d_plot(np_img):
#     plt.imshow(np_img, cmap='gray')   # 'gray' colormap forces grayscale display
#     plt.axis('off')                 # hide axis for clean view
#     plt.show()

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [8]:

# model = models.Sequential([
#     layers.Input(shape=(480, 720, 1)),

#     # Conv block 1
#     layers.Conv2D(64, kernel_size=3, strides=2, padding='valid', activation='relu'),
#     layers.MaxPooling2D(pool_size=4, strides=4),

#     # Conv block 2
#     layers.Conv2D(64, kernel_size=3, strides=2, padding='valid', activation='relu'),
#     layers.MaxPooling2D(pool_size=4, strides=4),

#     # Fully connected
#     layers.Flatten(),
#     layers.Dense(2, activation='softmax')
# ])

from tensorflow.keras.models import load_model

model_path = "my_model.keras"
model = load_model(model_path)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 239, 359, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 59, 89, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 44, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 11, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4928)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         9,858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,280 (555.79 KB)

 Trainable params: 47,426 (185.26 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 94,854 (370.53 KB)

In [9]:
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [10]:
def fit_seq_model():
    for _ in range(5):
        for dir in os.listdir("mv_data"):
            data = np.load("mv_data/" + dir + '/data.npz')
            Y = np.array([[0,1] if x==1 else [1,0] for x in data['Y']] )
            X = data['X']
            model.fit(X,Y,epochs=5,batch_size=64)

            print("done for batch : ",dir)
        model.save("my_model.keras")
        print("model saved for ",_)
        print("model saved for ",_)
        print("model saved for ",_)

In [11]:
def fit_random_model():
    dirs_list = [dir for dir in os.listdir("mv_data")]
    for num_times in range(5):
        for _ in range(len(dirs_list)):
            dir = dirs_list[np.random.randint(low=0,high=len(dirs_list))]
            print("train for: ",dir)
            data = np.load("mv_data/" + dir + '/data.npz')
            Y = np.array([[0,1] if x==1 else [1,0] for x in data['Y']] )
            X = data['X']
            model.fit(X,Y,epochs=5,batch_size=64)

            print("done for batch : ",dir)
        model.save("my_model.keras")
        print("model saved for ",_)
        print("model saved for ",_)
        print("model saved for ",_)

In [ ]:
class CNN():
    def __init__(self):
        model_path = "my_model.keras"
        self.model = load_model(model_path)
        

In [12]:
fit_random_model()

train for:  batch_2
Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 19s 973ms/step - accuracy: 0.9553 - loss: 0.1762
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 942ms/step - accuracy: 0.9642 - loss: 0.0993
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 949ms/step - accuracy: 0.9884 - loss: 0.0448
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 931ms/step - accuracy: 0.9841 - loss: 0.0571
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 936ms/step - accuracy: 0.9827 - loss: 0.0553
done for batch :  batch_2
train for:  batch_7
Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 997ms/step - accuracy: 0.9779 - loss: 0.0662
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 972ms/step - accuracy: 0.9910 - loss: 0.0316
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 960ms/step - accuracy: 0.9876 - loss: 0.0356
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 965ms/step - accuracy: 0.9883 - loss: 0.0311
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 966ms/step - accuracy: 0.9883 - loss: 0.0298
done for batch :  batch_7
train for:  batch_9
Epoch 1/5
12/12 ━━━━━━━━━━━━

In [13]:
# import numpy as np
# from tensorflow.keras.models import load_model
# data = np.load("mv_data/batch_5/data.npz")
# Y = np.array([[0,1] if x==1 else [1,0] for x in data['Y']] )
# X = data['X']
# model = load_model("my_model.keras")
# model.evaluate(X,Y)

In [16]:
def eval():
    for dir in os.listdir("mv_data"):
        data = np.load("mv_data/" + dir + '/data.npz')
        Y = np.array([[0,1] if x==1 else [1,0] for x in data['Y']] )
        X = data['X']
        model.evaluate(X,Y)

In [17]:
eval()

29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9636 - loss: 0.0842
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 0.9616 - loss: 0.1484
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.9744 - loss: 0.0486
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 0.9854 - loss: 0.0580
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.9934 - loss: 0.0378
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9814 - loss: 0.0487
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.9888 - loss: 0.0209
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 122ms/step - accuracy: 0.9738 - loss: 0.0667
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.9585 - loss: 0.0769
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.9686 - loss: 0.0739
